In [108]:
import pandas as pd
import numpy as np
import tensorflow as tf
import neattext.functions as nfx
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [109]:
data=pd.read_csv('emotion_dataset_raw.csv')

In [110]:
data['Clean Text']= data['Text'].apply(nfx.remove_userhandles)
data['Clean Text']= data['Clean Text'].apply(nfx.remove_stopwords)


In [111]:

X_features = data['Clean Text']
ylabel = data['Emotion']

In [112]:

# Initialize the tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_features)

# Convert text to sequences
X_sequences = tokenizer.texts_to_sequences(X_features)

# Pad sequences to ensure they have the same length
X_padded = pad_sequences(X_sequences, padding='post')
x_train,x_test,y_train,y_test = train_test_split (X_padded,ylabel,test_size=0.2,random_state=42)
# Encode target labels with value between 0 and n_classes-1.
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


In [113]:
model = Sequential([
    
    Dense(50,activation= 'relu'),
    Dense(25,activation='relu'),
    Dense(15,activation='relu'),
    Dense(1,activation='linear')
    
],
  name= 'model'
  )


In [118]:

model.compile(loss='mse',
              optimizer='adam')

model.fit(x_train, y_train_encoded, epochs=20, verbose=0)
mean_squared_error(y_test_encoded, model.predict(x_test))


218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


3.882258843075598

In [115]:
import numpy as np

# Reshape the input to match the expected shape
input_data = np.expand_dims(x_test[43], axis=0)
prediction = model.predict(input_data)

# Round the prediction to the nearest integer
predicted_label = np.round(prediction).astype(int)

# Ensure the predicted label is within the range of the labels
predicted_label = np.clip(predicted_label, 0, len(label_encoder.classes_) - 1)

# Inverse transform to get the original label
original_label = label_encoder.inverse_transform(predicted_label)

# Create a reverse mapping from indices to words
reverse_word_index = {value: key for key, value in tokenizer.word_index.items()}

# Convert the sequence back to text
original_text = ' '.join([reverse_word_index.get(i, '') for i in x_test[43]])

print(original_label)
print(original_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
['neutral']
coming home tmrw dont baby till sunday night                                                                      


c:\Users\a2\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
